# Banking Project

***

>The bank wants to improve their services. For instance, the bank managers have only vague idea, who is a good client (whom to offer some additional services) and who is a bad client (whom to watch carefully to minimize the bank loses). Fortunately, the bank stores data about their clients, the accounts (transactions within several months), the loans already granted, the credit cards issued. The bank managers hope to improve their understanding of customers and seek specific actions to improve services. A mere application of a discovery tool will not be convincing for them.  

>To test a data mining approach to help the bank managers, it was decided to address two problems, a descriptive and a predictive one. While the descriptive problem was left open, the predictive problem is the prediction of whether a loan will end successfuly.

> _ - in Banking Case Description, ECAC Moodle Page_

***

[Kaggle Challenge Page](https://www.kaggle.com/)

The steps performed are as follows:
* Data Loading and Cleaning
* Descriptive Data Mining & Feature Engineering
* Predictive Data Mining

## Tools

For this work we will use the common tools in a data scientist and engineer arsenal. All of them work together in a seamless fashion, as well as with the Jupyter Notebook (this enhanced interactive document).

* **Numpy** is the fundamental package for scientific computing with Python
* **Pandas** provides high-performance, easy-to-use data structures (_e.g._ data frames) and data analysis tools
* **Matplotlib** implements plotting functionality
* **Scikit Learn** aggregates advanced machine learning tools

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import sklearn

plt.style.use('ggplot')
%matplotlib inline

## 1. Data Loading and Cleaning

A key initial step in every data mining work is to clean the data. This reduces the occurence of future unexpected behaviors and gives a preliminary insight over the "raw" data.

The **transactions** records describe transactions on accounts, representing dynamic characteristics of the accounts.

In [ ]:
transactions_df = pd.read_csv('./data/banking - transaction.csv', 
                              sep=';',
                              parse_dates=['date'],
                              infer_datetime_format=True,
                              dtype={'bank':np.str},
                              index_col='trans_id')

In [ ]:
transactions_df.head()

**k_symbol** name is not very represent representative.

In [ ]:
transactions_df = transactions_df.rename(columns={
    'k_symbol': 'trans_char',
    'date': 'date created'
    })

In [ ]:
transactions_df.head()

The **accounts** records contain static characteristics of the accounts.

In [ ]:
accounts_df = pd.read_excel('./data/banking.xlsx', 
                            sheetname='account',
                            parse_dates=['date'],
                            infer_datetime_format=True,
                            index_col='account_id'
                           )

In [ ]:
accounts_df.head()

The **clients** records describe static characteristics of the clients.

In [ ]:
clients_df = pd.read_excel('./data/banking.xlsx',
                           sheetname='client',
                           index_col='client_id')

In [ ]:
clients_df.head()

The **birth_number** feature is not readable in this representation. We have, then, to parse it and transform it into two new columns: **birthday** and **gender**.

In [ ]:
clients_df['gender'] = clients_df.apply(lambda c: 'Male' if c['birth_number'] % 10000 < 5000 else 'Female', axis=1)

In [ ]:
from datetime import date

def normalize_birth_number(client):
    birth_number = int(client['birth_number'])
    year = birth_number // 10000
    month = (birth_number // 100) % 100
    day = birth_number % 100
    
    month = month if month < 50 else month - 50
    
    return  "{0:02d}{1:02d}{2:02d}".format(year, month, day)


clients_df['birth_number'] = clients_df.apply(normalize_birth_number, axis=1) # month - 50 on females
clients_df['birthday'] = pd.to_datetime(clients_df['birth_number'], format='%y%m%d')
clients_df['birthday'] = clients_df.apply(
    lambda c: c['birthday'] if c['birthday'].date() <= date.today() else (c['birthday'] - pd.tseries.offsets.DateOffset(years=100)), 
    axis=1) # if infered year > 2015 the it is in the 19's
clients_df = clients_df.drop('birth_number', axis=1)

In [ ]:
clients_df.head()

The **dispositions** records relate a client with an account (being useful in join operations).

In [ ]:
dispositions_df = pd.read_excel('./data/banking.xlsx',
                                sheetname='disposition',
                                index_col='disp_id')

In [ ]:
dispositions_df.head()

The **payment_orders** records, like **transaction** records, represent another dynamic characteristic of accounts.

In [ ]:
payment_orders_df = pd.read_excel('./data/banking.xlsx',
                                  sheetname='payment order',
                                  index_col='order_id')

In [ ]:
payment_orders_df.head()

The **loans** records describe information of a loan for an account.

In [ ]:
loans_df = pd.read_excel('./data/banking.xlsx',
                         sheetname='loan',
                         parse_dates=['date'],
                         infer_datetime_format=True,
                         index_col='loan_id')

In [ ]:
loans_df.head()

The **credit_cards** records describes static information of a credit card.

In [ ]:
credit_cards_df = pd.read_excel('./data/banking.xlsx',
                                sheetname='credit card',
                                parse_dates=['issued'],
                                infer_datetime_format=True,
                                index_col='card_id')

In [ ]:
credit_cards_df.head()

The **districts** records provide demographic information about a district.

In [ ]:
districts_df = pd.read_excel('./data/banking.xlsx',
                             sheetname='district',
                             index_col='A1')

In [ ]:
districts_df.head()

The column labels provided lack any useful information.

In [ ]:
districts_df = districts_df.rename(columns={
        'A2': 'district_name',
        'A3': 'region',
        'A4': 'no_inhabitants',
        'A5': 'no_municipalities_w_inhabitants_<499',
        'A6': 'no_municipalities_w_inhabitants_500-1999',
        'A7': 'no_municipalities_w_inhabitants_2000-9999',
        'A8': 'no_municipalities_w_inhabitants_>10000',
        'A9': 'no_cities',
        'A10': 'ratio_urban_inhabitants',
        'A11': 'average_salary',
        'A12': 'unemployment_rate_95',
        'A13': 'unemployment_rate_96',
        'A14': 'no_enterpreneurs_per_1000_inhabitants',
        'A15': 'no_commited_crimes_95',
        'A16': 'no_commited_crimes_96',
    })

districts_df.index.name = 'district_id'

In [ ]:
districts_df.head()

Lets check if the types infered by Pandas library are correct:

In [ ]:
districts_df.dtypes

We see that **unemployment_rate_95** and **no_commited_crimes_95** are loaded as objects.

In [ ]:
districts_df['unemployment_rate_95'].unique()

In [ ]:
districts_df['no_commited_crimes_95'].unique()

We see that both use a question mark to demark missing values. We'll convert properly those columns:

In [ ]:
districts_df['unemployment_rate_95'] = pd.to_numeric(districts_df['unemployment_rate_95'], errors='coerce')
districts_df['no_commited_crimes_95'] = pd.to_numeric(districts_df['no_commited_crimes_95'], errors='coerce')

districts_df.dtypes

## 2. Descriptive Data Mining & Feature Engineering

This first section aims at providing ways to better understand and extract value from the data. This is mostly accoplished by gathering descriptive statistics and ploting.

Considering this gathered knowledge, the datasets are edited and joined into useful intermediate format, which represent the main entities in the data, and then in a format in which the machine learning algorithms are able to understand (most of the times a single matrix, and most of the times without missing values).

The **loans** relate to the remainder entities through the **account** they are linked to. Therefore, the remainder entities should be summarized in such a way that each of the **accounts** information is given in a single row.

But before making any assumption, we should extract simple statistics about the feature space.

### Transactions Dataframe

In [ ]:
transactions_df.describe(include='all')

**type**, **operation** and **trans_char** seem all to represent the same information. Lets evaluate that:

In [ ]:
print("type:", transactions_df['type'].unique())

In [ ]:
print("operation:", transactions_df['operation'].unique())

In [ ]:
print("trans_char:", transactions_df['trans_char'].unique())

**operation** seems irrelevant give **type**:

In [ ]:
transactions_df_e = transactions_df.drop('operation', axis=1).copy() # 'e' for edited

Its also irrelevant the distinction between *withrawal* and *withrawal in cash*.

In [ ]:
mask = transactions_df_e['type'] == 'withdrawal in cash'
transactions_df_e.ix[mask, 'type'] = ('withdrawal')

We will create an additional column to store the signed amount (given by the type of operation), and another with the normalized **signed_amount** value, according to the **balance** previous to the operation (if an operation is the first one, we store 0):

In [ ]:
transactions_df_e['signed_amount'] = transactions_df_e.apply(lambda x: - x['amount'] if x['type'] == 'withdrawal' else x['amount'], axis=1)
transactions_df_e['norm_signed_amount'] = transactions_df_e.apply(lambda x: 
                                                                      0 if (x['balance'] - x['signed_amount']) == 0 
                                                                      else x['signed_amount'] / (x['balance'] - x['signed_amount']), 
                                                                  axis=1)

From **trans_char** we are able to extract if the user is pensionist, or if the user has been sanctioned for negative balance, among other things. We will create an additional table with that information, with the values weighted by the **amount**, indexed by **account_id**:

In [ ]:
# select the needed rows from transactions_df
trans_temp_df = transactions_df_e[['account_id', 'trans_char', 'norm_signed_amount']].copy()

# remove the rows where an empty string is present
mask = trans_temp_df.trans_char != ' '
trans_temp_df = trans_temp_df.ix[mask]

# remove the rows containing NaN
trans_temp_df = trans_temp_df.dropna(axis='index')

In [ ]:
# create the dataframe indexed by account_id
account_features_df = trans_temp_df[['account_id']].drop_duplicates(subset=['account_id'])
account_features_df = account_features_df.set_index('account_id')

# create the count columns, corresponding to the data countained in
    # pension
    # interest credited
    # household
    # statement
    # insurance payment
    # sanction for negative balance
    # loan payment

def create_trans_count_col(df, val):
    new_df = df.ix[df['trans_char'] == val].groupby('account_id').sum()
    new_df = new_df.rename(columns={'norm_signed_amount':val})
    return new_df

additional_dfs = [create_trans_count_col(trans_temp_df, val) for val in trans_temp_df['trans_char'].unique()]
account_features_df = account_features_df.join(additional_dfs)

account_features_df.head()

Now merge it with the **accounts** dataframe:

In [ ]:
accounts_df = accounts_df.join(account_features_df)
accounts_df.head()

Now we'll look into the **normalized signed amount** and **type** of each operation.

We will extract, for each **account**, the *count*, *mean* and *standard deviation* of operation values, as well as *mean* and *standard deviation* of the number of days between each **operation**, grouped by operation **type**.

In [ ]:
temp_df = transactions_df_e[['account_id', 'type', 'norm_signed_amount', 'date']].copy()

# create columns with dates converted to days since 01-01-1970
#temp_df['date_days'] = temp_df.apply(lambda x: (x['date'] - pd.datetime(1970,1,1)).days, axis=1)

In [ ]:
# sort firstly by account_id, then by date
temp_df = temp_df.sort_values(by=['account_id', 'date'])

#obtain, by row, the previous date
prev_dates = temp_df.groupby('account_id').apply(lambda x: x['date'].shift().fillna(x.iloc[0]['date'])).reset_index(level=0)
delta = temp_df['date'] - prev_dates['date']
delta = delta.astype("timedelta64[D]")
temp_df['date_delta'] = delta

In [ ]:
#temp_df = temp_df.drop('date', axis='columns')

def summarize_transactions(df):
    summaries_dfs = []
    for tp in df['type'].unique():
        tmp_df = temp_df[temp_df['type'] == tp].drop('type', axis=1)
        
        tmp_grp_df = tmp_df.groupby('account_id').agg([np.count_nonzero, np.average, np.std])
        
        ops_df = tmp_grp_df['norm_signed_amount']
        ops_df = ops_df.rename(columns={'count_nonzero': tp + '_cnt',
                                        'average': tp + '_avg',
                                        'std': tp + '_std',
                                       })
        
        dates_df = tmp_grp_df['date_delta']
        dates_df = dates_df.drop('count_nonzero', axis='columns')
        dates_df = dates_df.rename(columns={'average': tp + '_dates_avg',
                                            'std': tp + '_dates_std',
                                           })
        joined_summary_df = ops_df.join(dates_df)
        summaries_dfs.append(joined_summary_df)
    
    # now concatenate the summaries_dfs
    summaries_df = pd.concat(summaries_dfs, axis='columns')
    return summaries_df

trans_summaries_df = summarize_transactions(temp_df)
trans_summaries_df.head()

Now merge with **accounts** dataframe:

In [ ]:
accounts_df = accounts_df.join(trans_summaries_df)
accounts_df.head()

### Clients

Now we'll take a look at the features that characterize each user.

In [ ]:
clients_df.describe(include='all')

As for each user corresponds a district, we'll look into the corresponding table:

In [ ]:
districts_df.describe(include='all')

We can consider both the individual districts or the regions. 
In order to evaluate if the generalization for regions is reduces or not some interesting events, we'll use PCA to reduce the dimensionality of the table to two dimensions, therefore making it possible to plot.

In [ ]:
districts_df_e = districts_df.drop(['district_name', 'region'], axis='columns')
districts_df_e = districts_df_e.fillna(0)

from sklearn.decomposition import PCA
pca = PCA(n_components=2)
districts_reduced = pca.fit_transform(districts_df_e.values)

In [ ]:
districts_df_e['a'] = districts_reduced[:, 0]
districts_df_e['b'] = districts_reduced[:, 1]

# convert regions to integers, to use them as colors
regions = districts_df.apply(lambda x: x['region'][1], axis='columns').astype(int)

districts_df_e.plot(kind='scatter', x='a', y='b', c=regions)

From the plot we conclude that probably it is beneficial to work with districts.

In [ ]:
clients_districts_df = clients_df.join(districts_df.drop(['region', 'district_name'], axis='columns'))

# we no longer need district_id column
clients_districts_df = clients_districts_df.drop('district_id', axis='columns')

clients_districts_df.head()

Now we'll merge **Credit Cards** and **Clients** information with the accounts.

As there may be multiple credit cards and clients associated with a given account, we have to devise a certain heuristics.

In [ ]:
dispositions_df.describe(include='all')

In [ ]:
dispositions_df.type.unique()

To start, we'll simply use the information about the owner of the account.

We'll check the data if there is really only one owner per account.

In [ ]:
dispositions_owners_df = dispositions_df[dispositions_df['type'] == 'OWNER'].drop('type', axis='columns')

In [ ]:
owners_per_account_df = dispositions_owners_df.groupby(['account_id']).count()

owners_per_account_df[owners_per_account_df['client_id'] != 1].count(axis='index')

We can now merge **clients** with **accounts**

In [ ]:
accounts_clients_df = accounts_df.join(dispositions_owners_df).join(clients_districts_df) \
                      .drop(['account_id', 'client_id'], axis='columns')

accounts_clients_df.head()

### Loans

We want to keep a summary of the history of loans related to a given account.

In [ ]:
loans_df.describe(include='all')

In [ ]:
print(loans_df.status.unique())

For reference:
 * **A-** contract finished, no problems
 * **B-** contract finished, loan not payed
 * **C-** running contract, ok so far
 * **D-** running contract, client in debt
 
Lets  see how the loans distribute accross the four categories:

In [ ]:
loans_df.groupby('status').count()['account_id']

There are also a lot of contracts running, but most importantly there is a gret disparity between the two counts of the different loans results. We must be aware of this when training the algorithms. As we want to identify properly the loans that will have the **B** status, we have to be carefull when structuring the training and testing datasets.

In order to summarize this data, we want to obtain the *count* of loans in each state, as well as the *average* and *standard deviation* of the amount and duration associated with each loan.

In [ ]:
def summarize_loans(df):
    summaries_dfs = []
    for tp in df['status'].unique():
        tmp_df = df[df['status'] == tp].drop('status', axis=1)
        
        tmp_grp_df = tmp_df.groupby('account_id').agg([np.count_nonzero, np.average, np.std])
        
        amount_df = tmp_grp_df['amount']
        amount_df = amount_df.rename(columns={'count_nonzero': tp + '_cnt',
                                           'average': tp + '_amount_avg',
                                           'std': tp + '_amount_std',
                                          })
        
        duration_df = tmp_grp_df['duration']
        duration_df = duration_df.drop('count_nonzero', axis='columns')
        duration_df = duration_df.rename(columns={'average': tp + '_duration_avg',
                                                  'std': tp + '_duration_std',
                                                 })
        joined_summary_df = amount_df.join(duration_df)
        summaries_dfs.append(joined_summary_df)
    
    # now concatenate the summaries_dfs
    summaries_df = pd.concat(summaries_dfs, axis='columns')
    return summaries_df


loans_summary_df = summarize_loans(loans_df)
loans_summary_df.head()

And now merge with accounts:

In [ ]:
accounts_loans_df = accounts_clients_df.join(loans_summary_df)
accounts_loans_df.head()

In [ ]:
accounts_loans_df.dtypes

We've gathered 55 features to characterize each account, gathering information about the accounts, users (and respective credit cards), transactions and history of loans.

## 3. Predictive Data Mining

The above-mentioned accounts-indexed dataframe, combined with the specification of each loan (duration and amount), constitutes all the needed information to train our algorithms.


Firstly we have to prepare a dataframe such that the machine-learning algorithms are able to undertand it. In order to accomplish that, we will merge this **accounts_loans_df** dataframe with the original **loans_df** dataframe. For new loans that arive, the process will be the same, therefore we want to encapsulate the process in a function:

In [ ]:
accounts_final_df = accounts_loans_df

def join_loans_accounts(l_df):
    return l_df.join(accounts_final_df, lsuffix='new_loan_', rsuffix='account_')

loans_df.join(accounts_final_df)